# 基于字符的文本生成
给定一个字符，RNN自动根据该字符生成后续的文本.


*   此模型是基于字符的。训练开始时，模型不知道如何拼写一个英文单词，甚至不知道单词是文本的一个单位。
*   输出文本的结构类似于剧本 -- 文本块通常以讲话者的名字开始；而且与数据集类似，讲话者的名字采用全大写字母。
*   如下文所示，此模型由小批次 （batch） 文本训练而成（每批 100 个字符）。即便如此，此模型仍然能生成更长的文本序列，并且结构连贯。



In [ ]:
# import reposity
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
# download dataset 
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# load the dataset
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
vocab = set(text)
vocab = sorted(vocab)
print(vocab)
# print vocab之后可以发现莎士比亚写的文章里面字符其实没有多少23333

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# 创建从vocab到其索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
print("索引映射：")
print(char2idx, '\n')

idx2char = np.array(vocab)
print(idx2char,'\n')

# 创建了index和char的映射，然后可以将整个text转化为由index数字代表。
text_as_int = np.array([char2idx[c] for c in text])
print("转化为int的text文档前100个字符:")
print(text_as_int[:100])

索引映射：
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64} 

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z'] 

转化为int的text文档前100个字符:
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 

## 构造模型

使用RNN训练，每次输入长度为100的字符，以此的训练结果来预测：当给定一个字符时，下一个字符是什么。

In [ ]:
# 设定输入长度
seq_length = 100

#examples_per_epoch = len(text) // seq_length 

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

print(char_dataset)

<TensorSliceDataset shapes: (), types: tf.int64>


join: str.join(sequence)，将sequence序列中元素以str为间隔生成新字符串。

repr: 返回一个对象的 string 格式。

In [ ]:
# batch 函数，把Dataset切分成seq_length大小的部分，drop_remainder代表了是否drop最后多余部分
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
def split_input_target(text):
  input_text = text[:-1]
  target_text = text[1:]
  return input_text, target_text

# input_text 输入数据包含整个字符串
# target_text 输出目标不包括第一个字符，这是和该问题的设计相关的，
# 问题设计中，要求输入一个字符，生成由此字符衍生的一系列字符，生成的部分是不包括事先输入的首字符的，因此target_text中不包括。

dataset = sequences.map(split_input_target)
# show dataset
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

print(dataset)

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


# 创建模型


1.   使用了字词嵌入，word_embedding, 其将每个字符的数字映射到一个embedding_dim维度的向量中。
2.   GRU，简化版的RNN类型，大小由units=rnn_units制定（教程说用LSTM也可以）
3.   Dense 全连接层



In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # 三层网络
    # 第一层embedding，对于每个字符，生成一个embedding，
    # 第二层GRU（RNN），把embedding输入到RNN，rnn_units定义为1024,
    # 第三层Dense 输入字符预测.
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  model.summary()
  return model

# invoke build_model 
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size = BATCH_SIZE
)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 损失函数

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# from_logits=True是因为模型返回逻辑回归
model.compile(optimizer='adam', loss=loss)

# 设置检查点
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 24s 137ms/step - loss: 2.7138
Epoch 2/10
172/172 [==============================] - 23s 136ms/step - loss: 1.9675
Epoch 3/10
172/172 [==============================] - 23s 136ms/step - loss: 1.7005
Epoch 4/10
172/172 [==============================] - 23s 136ms/step - loss: 1.5498
Epoch 5/10
172/172 [==============================] - 23s 136ms/step - loss: 1.4608
Epoch 6/10
172/172 [==============================] - 23s 136ms/step - loss: 1.3991
Epoch 7/10
172/172 [==============================] - 23s 136ms/step - loss: 1.3534
Epoch 8/10
172/172 [==============================] - 23s 136ms/step - loss: 1.3143
Epoch 9/10
172/172 [==============================] - 23s 136ms/step - loss: 1.2791
Epoch 10/10
172/172 [==============================] - 23s 136ms/step - loss: 1.2459


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [34]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [42]:
def generate_text(model, start_string):
  # 根据一个预设的字符串来生成text

  # 要生成的字符个数
  num_generated = 1000
  # 将起始字符转化为数字(向量化)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  # 为什么要expand_dims

  # 生成的结果
  text_generated = []

  # 低温度会生成更可预测的文本
  # 较高温度会生成更令人惊讶的文本
  # 可以通过试验以找到最好的设定
  # 在这个demo里，temperature就是1

  temperature = 1.0

  model.reset_states
  for i in range(num_generated):
    #print(input_eval.shape)
    predictions = model(input_eval)
    # 删除批次的维度
    predictions = tf.squeeze(predictions, 0)
    # 为什么要squeeze？ A：这里BATCH_SIZE为1，所以prediction格式为(1,None,65),
    #squeeze(_,0)可以删除维度1

    # 用分类分布预测模型返回的字符
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
    input_eval = tf.expand_dims([predicted_id], 0)
    #print(input_eval)
    text_generated.append(idx2char[predicted_id])
  
  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="Shuzhi"))



Shuzhing ouest lose his son Buckingham,
Like euple, royal friends, am I tready,
No, I'll not fall thee to my dreadful man.

LADY ANNE:
I am said we know, thy shame, most grain; and let me disperse.

LADY ANNE:
And beast, yet me purpose. What say, ye; I beseech you, you know not am farewell,
for the boy, in that they sees, the glass.
Lo, right tenderous; arroy, where the lies, adventuriouse.

LUCENTIO:
Her speed so ill-special, and what cheek and best before.

KING HENRY VI:
And then of plant, win your good shippress, or in a day;
And his mag speak thrself and make
His reverend both by destray'd
To hingless heavy,
Pername hath believe me
To you.

LUCIO:
Which quick ly a Pauliny, know excease leave.

GLOUCESTER:
Against the other gapple coponderitation,
And mailty of honour than we did cirly teach you; for that
Even Rens of the price; o' cack him, and
make curnish scander by the highness on thy form,
But severe they now and old earls break now,
No less of us, the liling their heads of th

一开始我感到困惑，generate_text函数中描述将预测出的字符和隐藏状态一起传递给模型作下一次输入。

实际中传给model的只是一个新生成的字符，并且考虑到我们读取了训练好的模型参数，所以模型的参数也不会再改变。但是：

model中keras.layers.GRU设置了一个RNN层，RNN内部包含context信息，所以在generate_text函数中，第一次迭代和第二次迭代时context是不同的，所以隐藏状态一直在变化。

# 如何摆脱model.fit函数的限制自定义训练过程？